# Packages and Initial Setup

In [ ]:
! pip install openai datasets together

In [ ]:
import json
import requests
import random
import time
from datasets import load_dataset
import pandas as pd
import together
from openai import OpenAI
import os
import csv
from tqdm import tqdm

TOGETHER_API_KEY = # this key stems from togetherAI.
GPT_API_KEY = # this key stems from openAI.

In [ ]:
# model_name = "meta-llama/Llama-3-70b-chat-hf"
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "lmsys/vicuna-7b-v1.5"
# model_name = "Qwen/Qwen1.5-7B-Chat"
# model_name = "QWEN/QWEN1.5-72B-CHAT"
# model_name = "gpt-3.5-turbo-0125"

# Load Datasets - TATQA, WikiTQ and HybridQA

In [ ]:
# TATQA
random.seed(42)
file_path = ''#TAT-QA-train-markdown.json
with open(file_path, 'r') as file:
    dataset_TAT = json.load(file)
test = random.sample(dataset_TAT, 500)

def extract_questions(text_list):
    questions = []
    for text in text_list:
        # Find the start of the question part
        question_index = text['input'].find('\nQuestion:\n')
        if question_index != -1:
            # Extract everything after 'Question:\n'
            question = text['input'][question_index:]
            questions.append(question)
    return questions

def extract_tables(x):
  tables = []
  start_marker = "Table:\n"
  end_marker = "\nQuestion:\n"
  for item_input in x:
    end_index = item_input['input'].find(end_marker)
    tables.append(item_input['input'][len(start_marker)+1:end_index])
  return tables

questions = extract_questions(test)
tables = extract_tables(test)
test = pd.DataFrame(test)
test['Question'] = questions
test['Question'] = test['Question'].str.replace("\nQuestion:\n", "", regex=False)
test['Table']  = tables
test = test[['Question','Table','output']]
test.rename(columns={'output': 'True'}, inplace=True)

In [ ]:
# WikiTQ
dataset = load_dataset("wikitablequestions", split = 'test', trust_remote_code=True)
random_seed = 42
df = pd.DataFrame(dataset)
test = df.sample(n = 500, random_state = random_seed)
test = test.reset_index(drop=True)
test.rename(columns={'question': 'Question', 'answers': 'True', 'table': 'Table'}, inplace=True)
test.head()

In [ ]:
# HybridQA
test = pd.read_csv('') # hybrid_4096.csv
test.rename(columns={'question': 'Question', 'table': 'Table', 'answer_text': 'True'}, inplace=True)

# GPT4 Evaluator

In [ ]:
def eval_w_gpt4(f):
  client = OpenAI(api_key = GPT_API_KEY)
  system = '''You are an intelligent assessment assistant.'''
  prefix = '''Based on the question and the golden answer, judge whether the predicted answer correctly answers the question and give only a Yes or No.\n'''

  df = pd.read_csv(f)

  for i in range(len(df)):
    question = df.loc[i, "Question"]
    answer = df.loc[i, "True"]
    prediction = df.loc[i, "Prediction"]

    prompt = prefix + f"Question: [{question}]\nGolden answer: [{answer}]\nPredicted answer: [{prediction}]"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
          {"role": "system", "content": system},
          {"role": "user", "content": prompt},
        ]
      )
    output = response.choices[0].message.content
    df.loc[i, "GPT4_result"] = output
    print(i, output)

  num_of_yes = df['GPT4_result'].value_counts()['Yes']
  acc = (num_of_yes / len(df))
  print("Accuracy:", acc)
  df.to_csv(f, index=False)

# Baseline Vanilla

In [ ]:
model_name = # model name
file_name = # saved file path
start = 0
max = len(test)
end = max

In [ ]:
for i in tqdm(range(start, end)):
  question = test.loc[i, "Question"]
  table = test.loc[i, "Table"]
  answer = test.loc[i, "True"]

  client = OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url='https://api.together.xyz/v1',
  )
  prompt = "{} Given this table, answer this question {}. You do not need to explain the answer.".format(table, question)

  try:
    chat_completion = client.chat.completions.create(
      messages=[
        {
          "role": "system",
          "content": "You handle table information well.",
        },
        {
          "role": "user",
          "content": f'''<bos><start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model''' if model_name == "google/gemma-7b-it" else prompt,
        }
      ],
      model=model_name
    )

    prediction = chat_completion.choices[0].message.content
    with open(file_name, 'a+', newline='') as file:
      writer = csv.writer(file)
      if os.path.getsize(file_name) == 0:
          writer.writerow(["Question", "Table", "Prediction", "True"])
      writer.writerow([question, table, prediction, answer])
  except:
    print("Error")
    continue

In [ ]:
eval_w_gpt4(file_name)

# Baseline Self-Augmentation

In [ ]:
model_name = # model name
file_name = # saved file path
start = 0
max = len(test)
end = 100

In [ ]:
for i in tqdm(range(start, end)):
  question = test.loc[i, "Question"]
  table = test.loc[i, "Table"]
  answer = test.loc[i, "True"]

  client = OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url='https://api.together.xyz/v1',
  )

  # First Phase: Generating Intermediate Structural Knowledge
  prompt_phase_1 = f'''{table} Analyze this table and identify the critical values, ranges, and necessary calculations needed to answer the question below:
  Question: {question}'''

  try:
    chat_completion_1 = client.chat.completions.create(
      messages=[
        {
          "role": "system",
          "content": "You handle table information well.",
        },
        {
          "role": "user",
          "content": f'''<bos><start_of_turn>user\n{prompt_phase_1}<end_of_turn>\n<start_of_turn>model''' if model_name == "google/gemma-7b-it" else prompt_phase_1,
        }
      ],
      model=model_name
    )

    intermediate = chat_completion_1.choices[0].message.content

    # Second Phase: Using the guidance to answer the question
    prompt_phase_2 = f'''{table} Given this table, answer this question: {question}. Additional information of the table: {intermediate}'''

    chat_completion_2 = client.chat.completions.create(
      messages=[
        {
          "role": "system",
          "content": "You handle table information well.",
        },
        {
          "role": "user",
          "content": f'''<bos><start_of_turn>user\n{prompt_phase_2}<end_of_turn>\n<start_of_turn>model''' if model_name == "google/gemma-7b-it" else prompt_phase_2,
        }
      ],
      model=model_name
    )

    prediction = chat_completion_2.choices[0].message.content

    with open(file_name, 'a+', newline='') as file:
      writer = csv.writer(file)
      if os.path.getsize(file_name) == 0:
          writer.writerow(["Question", "Table", "Prediction", "True"])
      writer.writerow([question, table, prediction, answer])
  except:
    print("Error")
    continue

In [ ]:
eval_w_gpt4(file_name)

# Table-Logic Sequential Prompting

In [ ]:
step1 = ''' Table: {table}\nIdentify critical columns highly relevant to this question: {question}. Be brief and short. Do NOT include answer.'''
step2 = ''' Table: {table}\nCritical columns: {column}\nBased on the critical columns of this table, identify critical rows highly relevant to this question: {question}. Be brief and short. Do NOT include answer.'''
step3 = ''' Table: {table}\nCritical columns: {column}\nCritical rows: {row}\nBased on the key values in critical columns and rows of this table, identify any aggregation, calculation, and comparison required by this question: {question}. Be brief and short. Do NOT include answer.'''
step4 = ''' Table: {table}\nGiven this table, answer this question: {question}. You do not need to explain the answer. Additional information that may help is given below. Critical columns: {column}\nCritical rows: {row}\nAggregation, calculation, and comparison: {aggregation}'''

In [ ]:
def inference(prompt, model_name):
  chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "system",
      "content": "You handle table information well.",
    },
    {
      "role": "user",
      "content": prompt,
    }
    ],
  model=model_name,
  max_tokens = 4096 # 500 for vicuna-7b and llama2-7b
  )

  result = chat_completion.choices[0].message.content
  return result

In [ ]:
from tqdm import tqdm
import csv
import os
file_name = "" # saved file path
model_name = "" # model name
for i in tqdm(range(len(df))):
  question = df.loc[i, "Question"]
  table = df.loc[i, "Table"]
  answer = df.loc[i, "True"]

  client = OpenAI(
      api_key=TOGETHER_API_KEY,
      base_url='https://api.together.xyz/v1',
  )

  # For GPT3.5 model:
  # client = OpenAI(
  #   api_key=OPEN_AI_API_KEY
  # )

  try:
    # ************ Step1 ************
    prompt1 = step1.format(table = table, question = question)
    column = inference(prompt1, model_name)

    # ************ Step2 ************
    prompt2 = step2.format(table = table, question = question, column = column)
    row = inference(prompt2, model_name)

    # ************ Step3 ************
    prompt3 = step3.format(table = table, question = question, column = column, row = row)
    aggregation = inference(prompt3, model_name)

    # ************ Step4 ************
    prompt4 = step4.format(table = table, question = question, column = column, row = row, aggregation = aggregation)
    prediction = inference(prompt4, model_name)

    with open(file_name, 'a+', newline='') as file:
        writer = csv.writer(file)
        if os.path.getsize(file_name) == 0:
            writer.writerow(["Question", "Table", "True", "Column", "Row", "Aggregation", "Prediction"])
        writer.writerow([question, table, answer, column, row, aggregation, prediction])
  except Exception as E:
    print(E)
    df.drop(index=i)

# Table-Logic Segment Experiment

In [ ]:
step1 = ''' Table: {table}\nGiven this table, ONLY focus on information of table in columns: {column} and rows: {row}\n Answer this question: {question}. You do not need to explain the answer. '''
step2 = ''' Table: {table}\nGiven this table, answer this question {question} STRICTLY follow the instruction of Aggregation, calculation, and comparison needed: {aggregation}. You do not need to explain the answer. '''

In [ ]:
def inference(prompt, model_name):
  chat_completion = client.chat.completions.create(
  messages=[
    {
      "role": "system",
      "content": "You handle table information well.",
    },
    {
      "role": "user",
      "content": prompt,
    }
    ],
  model=model_name,
  max_tokens = 4096 # 500 for vicuna-7b and llama2-7b
  )
  result = chat_completion.choices[0].message.content
  return result

In [ ]:
model_name = # model name
file_name = # saved file path

In [ ]:
df = pd.read_csv('') # load generated instruction dataset from other models

In [ ]:
from tqdm import tqdm
import csv
import os
for i in tqdm(range(len(df))):
  question = df.loc[i, "Question"]
  table = df.loc[i, "Table"]
  column = df.loc[i, "Column"]
  row = df.loc[i, "Row"]
  aggregation = df.loc[i,"Aggregation"]
  answer = df.loc[i, "True"]

  client = OpenAI(
      api_key=TOGETHER_API_KEY,
      # base_url='https://api.together.xyz/v1'if using togetherAI,
  )
  try:

    # Given critical columns and rows generated from other models
    prompt = step1.format(table = table, column = column, row = row, question = question)
    prediction = inference(prompt, model_name)

    # Given aggregation generated from other models
    # prompt = step2.format(table = table, question = question, aggregation = aggregation)
    # prediction = inference(prompt, model_name)

    # Given table instruction
    # TATQA instruction
    instruction = "Please read the following table in Markdown format and related paragraphs, and then answer the question according to the table and paragraphs. Table cells in one row are seperated by '|', and different rows are seperated by '\n'."

    # WikiTQ instruction
    # instruction = "Data is structured in a dictionary with keys 'header' for column names and 'rows' for data, formatted as lists under each key."

    # HybridQA instruction
    # instruction = "Data is organized in a dictionary with keys for URL, title, headers, and rows. Headers define column titles, and rows are lists containing cell data and optional URLs for detailed information."

    prompt = "{} {} Given this table, answer this question {}. You do not need to explain the answer.".format(instructuion, table, question)

    with open(file_name, 'a+', newline='') as file:
        writer = csv.writer(file)
        if os.path.getsize(file_name) == 0:
            writer.writerow(["Question", "Table", "True", "Prediction"])
        writer.writerow([question, table, answer, prediction])
  except Exception as E:
    print(E)
    df.drop(index=i)

In [ ]:
eval_w_gpt4(file_name)

# Value Location Experiment

## Col Row Retrieve

In [ ]:
import re

def find_tuple(string):
  pattern = r"\((-?\d+),\s*(-?\d+)\)"
  match = re.search(pattern, string)
  if match:
      tuple_result = (int(match.group(1)), int(match.group(2)))
      return tuple_result
  else:
      print("No tuple found.")

In [ ]:
model_name = # model name
start = 0
max = len(df)
end = max

file_name = '/content/drive/MyDrive/12-final_project/value_loc_gpt3.5_result.csv'

In [ ]:
for i in tqdm(range(start, end)):
  # question = test.loc[i, "Question"]
  table = df.loc[i, "Table"]
  # answer = test.loc[i, "True"]
  value = df.loc[i, "Value"]
  row = df.loc[i, "Row"]
  column = df.loc[i, "Column"]

  client = OpenAI(
    api_key=TOGETHER_API_KEY,
    base_url='https://api.together.xyz/v1',
  )
  prompt = '''{} \nGiven this table, provide the position of {} as a tuple, where the first number represents row index and the second number represents column index.
  You do not need to explain the answer.'''.format(table, value)
  system_message = "You handle table information well."

  try:
    chat_completion = client.chat.completions.create(
      messages=[
        {
          "role": "system",
          "content": "You handle table information well.",
        },
        {
          "role": "user",
          "content": prompt,
        }
      ],
      model=model_name,
      max_tokens=2048,
    )

    pred = chat_completion.choices[0].message.content
    rc = find_tuple(pred)
    with open(file_name, 'a+', newline='') as file:
      writer = csv.writer(file)
      if os.path.getsize(file_name) == 0:
          writer.writerow(["Table", "Value", "Row", "Column", "pred_row", "pred_column"])
      writer.writerow([table, value, row, column, rc[0], rc[1]])
  except Exception as E:
    print(E)
    continue

## Value Look-up

In [ ]:
def extract_value_from_brackets(text):
    start = text.find('[') + 1
    end = text.find(']', start)
    if start > 0 and end > 0:
        return text[start:end]
    else:
        return "Value not found"

In [ ]:
model_name = # model name

start = 0
max = len(df)
end = max

file_name = # saved file path

In [ ]:
for i in tqdm(range(start, end)):
  # question = test.loc[i, "Question"]
  table = df.loc[i, "Table"]
  # answer = test.loc[i, "True"]
  value = df.loc[i, "Value"]
  row = df.loc[i, "Row"]
  column = df.loc[i, "Column"]
  pred_row = df.loc[i, "pred_row"]
  pred_column = df.loc[i, "pred_column"]

  client = OpenAI(
    # api_key=TOGETHER_API_KEY,
    # base_url='https://api.together.xyz/v1',
    api_key=GPT_API_KEY,
  )

  prompt = f'''{table} \nGiven this table, provide the cell value at row index {row} and column index {column} in square brackets. Example: [cell value]
    You do not need to explain the answer.'''
  system_message = "You handle table information well."

  try:
    chat_completion = client.chat.completions.create(
      messages=[
        {
          "role": "system",
          "content": "You handle table information well.",
        },
        {
          "role": "user",
          "content": prompt,
        }
      ],
      model=model_name,
      max_tokens=2048,
    )

    pred = chat_completion.choices[0].message.content
    pred_val = extract_value_from_brackets(pred)

    with open(file_name, 'a+', newline='') as file:
      writer = csv.writer(file)
      if os.path.getsize(file_name) == 0:
          writer.writerow(["Table", "Value", "Row", "Column", "pred_row", "pred_col", "pred_val"])
      writer.writerow([table, value, row, column, pred_row, pred_column, pred_val])
  except Exception as E:
    print(E)
    continue

# Plot Radar Chart

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from math import pi

# Data
data = {
    'Model': ['Llama3-70B', 'Llama2-7B', 'Vicuna-7B', 'QWen-7B'],
    'Table Partition_Row': [0.44, 0.21, 0.08, 0.23],
    'Table Partition_Column': [0.94, 0.36, 0.21, 0.45],
    'Value Lookup': [0.40, 0.08, 0.05, 0.12],
    'Column Finding': [0.79, 0.11, 0.20, 0.31],
    'Row Finding': [0.52, 0.15, 0.14, 0.18]
}

df = pd.DataFrame(data)

# Number of variables
categories = list(df)[1:]
N = len(categories)

# What will be the angle of each axis in the plot?
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Initialise the radar plot
ax = plt.subplot(111, polar=True)

# Draw one axe per variable + add labels
plt.xticks(angles[:-1], categories)

# Draw ylabels
ax.set_rlabel_position(0)
plt.yticks([0.1,0.3,0.5,0.7,0.9], ["0.1","0.3","0.5","0.7","0.9"], color="grey", size=7)
plt.ylim(0,1)

# Plot each model
for index, row in df.iterrows():
    values = row.drop('Model').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=1, linestyle='solid', label=row['Model'])
    ax.fill(angles, values, alpha=0.1)

# Add a legend
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

plt.show()